<a href="https://colab.research.google.com/github/teevvr7/Football_penalty/blob/main/Small_MODEL(final_model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
import zipfile
import os

# Upload the zip file from your computer
uploaded = files.upload()

# Get the filename of the uploaded zip (it's a dictionary, this gets the first key)
zip_filename = next(iter(uploaded))

# Extract the zip file into the Colab environment
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall('./')

# List the contents to confirm it worked
data_dir = './penalty_data'
print("Contents of the data directory:")
print(os.listdir(data_dir))

Saving penalty_data.zip to penalty_data.zip
Contents of the data directory:
['NerArg10.mp4', 'NerArg6.mp4', 'NerArg4.mp4', 'FraArg5.mp4', 'BraCro3.mp4', 'FraArg6.mp4', 'JpnCro5.mp4', 'FraArg4.mp4', 'BraCro2.mp4', 'JpnCro6.mp4', 'JpnCro7.mp4', 'FraArg2.mp4', 'NerArg3.mp4', 'NerArg1.mp4', 'BraCro7.mp4', 'NerArg5.mp4', 'JpnCro2.mp4', 'FraArg3.mp4', 'labels.csv', 'BraCro5.mp4', 'MorSpa5.mp4', 'MorSpa4.mp4', 'NerArg8.mp4', 'BraCro8.mp4', 'BraCro1.mp4', 'NerArg2.mp4', 'MorSpa3.mp4', 'JpnCro1.mp4', 'JpnCro3.mp4', 'FraArg1.mp4', 'FraArg8.mp4', 'MorSpa1.mp4', 'BraCro6.mp4', 'MorSpa2.mp4', 'MorSpa6.mp4', 'NerArg7.mp4', 'MorSpa7.mp4', 'JpnCro8.mp4', 'NerArg9.mp4', 'BraCro4.mp4', 'JpnCro4.mp4', 'FraArg7.mp4']


In [3]:
# === COLAB-COMPATIBLE CODE ===
# This version works with Colab's default packages

# First, let's check what versions we have and install only what's missing
print("🔍 Checking current package versions...")

# Import first to see what's available
try:
    import pandas as pd
    print(f"✅ pandas: {pd.__version__}")
except:
    !pip install pandas
    import pandas as pd

try:
    import cv2
    print(f"✅ OpenCV: {cv2.__version__}")
except:
    !pip install opencv-python-headless
    import cv2

try:
    import sklearn
    print(f"✅ scikit-learn: {sklearn.__version__}")
except:
    !pip install scikit-learn
    import sklearn

try:
    import mediapipe as mp
    print(f"✅ mediapipe: {mp.__version__}")
except:
    !pip install mediapipe
    import mediapipe as mp

try:
    from ultralytics import YOLO
    print("✅ ultralytics: Available")
except:
    !pip install ultralytics
    from ultralytics import YOLO

# Now import the rest
import os
import math
import joblib
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from google.colab import files

print("✅ All packages imported successfully!")

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Load YOLO model for person detection
yolo_model = YOLO('yolov8n.pt')

def calculate_angle(a, b, c):
    """Calculates the angle at point b formed by points a, b, c."""
    a_vec = np.array([a.x - b.x, a.y - b.y])
    c_vec = np.array([c.x - b.x, c.y - b.y])
    dot_product = np.dot(a_vec, c_vec)
    magnitude = np.linalg.norm(a_vec) * np.linalg.norm(c_vec)
    if magnitude == 0:
        return 0
    angle_rad = np.arccos(np.clip(dot_product / magnitude, -1.0, 1.0))
    return np.degrees(angle_rad)

def find_shooter(frame, yolo_model):
    """Uses YOLO to find the person most likely to be the shooter"""
    results = yolo_model(frame, verbose=False, conf=0.3)

    people_boxes = []
    for result in results:
        if result.boxes is not None:
            for box in result.boxes:
                class_id = int(box.cls)
                if yolo_model.names[class_id] == 'person':
                    people_boxes.append(box.xyxy[0].cpu().numpy())

    if not people_boxes:
        return None

    frame_height, frame_width = frame.shape[:2]
    bottom_center = (frame_width // 2, frame_height)

    best_box = None
    min_distance = float('inf')

    for box in people_boxes:
        x1, y1, x2, y2 = box
        box_center = ((x1 + x2) / 2, (y1 + y2) / 2)
        distance = math.sqrt((box_center[0] - bottom_center[0])**2 +
                            (box_center[1] - bottom_center[1])**2)

        if distance < min_distance:
            min_distance = distance
            best_box = box

    return best_box

def smart_crop(frame, best_box, min_size=300):
    """Smart cropping with resizing for small detections"""
    x1, y1, x2, y2 = best_box.astype(int)

    # Add padding
    padding = 40
    x1 = max(0, x1 - padding)
    y1 = max(0, y1 - padding)
    x2 = min(frame.shape[1], x2 + padding)
    y2 = min(frame.shape[0], y2 + padding)

    cropped_frame = frame[y1:y2, x1:x2]

    if cropped_frame.size == 0:
        return frame

    # Resize if too small
    height, width = cropped_frame.shape[:2]
    if height < min_size or width < min_size:
        scale = max(min_size / height, min_size / width)
        new_height = int(height * scale)
        new_width = int(width * scale)
        cropped_frame = cv2.resize(cropped_frame, (new_width, new_height), interpolation=cv2.INTER_LINEAR)

    return cropped_frame

# Load your CSV file
print("📊 Loading dataset...")
csv_path = './penalty_data/labels.csv'
labels_df = pd.read_csv(csv_path)
data_dir = './penalty_data'

# Lists to store features and labels
feature_list = []
label_list = []
success_count = 0

print("Starting feature extraction from videos...")

# Process each video
for index, row in labels_df.iterrows():
    video_name = row['video_name']
    frame_num = row['frame_number']
    direction_label = row['shooter_direction']

    video_path = os.path.join(data_dir, video_name)

    if not os.path.exists(video_path):
        print(f"⚠️  Video {video_path} not found. Skipping.")
        continue

    # Read the specific frame
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
    success, frame = cap.read()
    cap.release()

    if not success:
        print(f"⚠️  Could not read frame {frame_num} from {video_name}")
        continue

    # Find shooter
    shooter_box = find_shooter(frame, yolo_model)
    if shooter_box is None:
        print(f"⚠️  No person detected in {video_name}")
        continue

    # Crop and process
    processed_frame = smart_crop(frame, shooter_box)

    # Pose detection
    with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.2) as pose:  # Lower confidence threshold
        frame_rgb = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        if not results.pose_landmarks:
            print(f"⚠️  No pose in {video_name}:{frame_num}")
            continue

        landmarks = results.pose_landmarks.landmark

        # Extract key features
        try:
            # Plant foot angles (most important features)
            l_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE]
            l_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE]
            l_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
            plant_left = calculate_angle(l_ankle, l_knee, l_hip)

            r_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE]
            r_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE]
            r_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP]
            plant_right = calculate_angle(r_ankle, r_knee, r_hip)

            # Body lean
            l_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
            r_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]
            l_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
            r_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP]

            shoulder_center = (l_shoulder.x + r_shoulder.x) / 2
            hip_center = (l_hip.x + r_hip.x) / 2
            body_lean = shoulder_center - hip_center

            # Foot forwardness
            foot_forwardness = l_ankle.y - r_ankle.y

            # Shoulder-hip angle
            shoulder_hip_angle = calculate_angle(l_shoulder, l_hip, l_knee)

        except (IndexError, AttributeError) as e:
            print(f"⚠️  Error extracting features: {e}")
            continue

        # Create feature vector (top 5 features only)
        features = [plant_left, plant_right, foot_forwardness, body_lean, shoulder_hip_angle]
        feature_list.append(features)
        label_list.append(direction_label)
        success_count += 1
        print(f"✅ {video_name}:{frame_num} -> {direction_label}")

print(f"\n📈 Successfully processed {success_count} out of {len(labels_df)} samples")

if success_count == 0:
    print("❌ No samples were processed successfully. Please check your video files.")
else:
    # Convert to arrays
    X = np.array(feature_list)
    y = np.array(label_list)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    print(f"Training set: {X_train.shape}, Test set: {X_test.shape}")

    # Train model
    print("🤖 Training K-Nearest Neighbors model...")
    model = KNeighborsClassifier(n_neighbors=3)
    model.fit(X_train, y_train)

    # Evaluate
    accuracy = model.score(X_test, y_test)
    print(f"🎯 Model accuracy: {accuracy:.3f}")

    # Save model
    model_filename = 'final_penalty_predictor.pkl'
    joblib.dump(model, model_filename)
    print(f"💾 Model saved as {model_filename}")

    # Download
    files.download(model_filename)
    print("📦 Model downloaded! Ready for deployment.")

print("\n=== PROCESS COMPLETE ===")

🔍 Checking current package versions...
✅ pandas: 2.2.2
✅ OpenCV: 4.11.0
✅ scikit-learn: 1.6.1
✅ mediapipe: 0.10.21
✅ ultralytics: Available
✅ All packages imported successfully!
📊 Loading dataset...
Starting feature extraction from videos...
⚠️  No pose in BraCro1.mp4:84
✅ BraCro2.mp4:55 -> right
✅ BraCro3.mp4:79 -> center
✅ BraCro4.mp4:142 -> left
✅ BraCro5.mp4:75 -> left
✅ BraCro6.mp4:79 -> left
✅ BraCro7.mp4:133 -> left
✅ BraCro8.mp4:143 -> left
⚠️  No pose in FraArg1.mp4:101
✅ FraArg2.mp4:127 -> left
⚠️  No pose in FraArg3.mp4:127
✅ FraArg4.mp4:90 -> center
✅ FraArg5.mp4:152 -> left
✅ FraArg6.mp4:269 -> left
✅ FraArg7.mp4:106 -> center
✅ FraArg8.mp4:177 -> left
✅ JpnCro1.mp4:94 -> right
✅ JpnCro2.mp4:32 -> left
✅ JpnCro3.mp4:86 -> left
✅ JpnCro4.mp4:40 -> center
✅ JpnCro5.mp4:89 -> right
✅ JpnCro6.mp4:70 -> left
✅ JpnCro7.mp4:42 -> left
✅ JpnCro8.mp4:72 -> left
✅ MorSpa1.mp4:44 -> right
✅ MorSpa2.mp4:77 -> right
✅ MorSpa3.mp4:117 -> center
✅ MorSpa4.mp4:147 -> right
✅ MorSpa5.mp4:1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📦 Model downloaded! Ready for deployment.

=== PROCESS COMPLETE ===


In [1]:
# === COMPLETE PENALTY PREDICTION MODEL TRAINING ===
# This code will process your videos, extract features, train the model, and save the .pkl file

# Install required packages
!pip install mediapipe==0.10.9 ultralytics==8.0.20 opencv-python-headless==4.8.0.74 scikit-learn==1.3.2 pandas==1.5.3 numpy==1.23.5 --no-cache-dir --force-reinstall --quiet

# Import libraries
import pandas as pd
import cv2
import os
import math
import joblib
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import mediapipe as mp
from ultralytics import YOLO
from google.colab import files

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Load YOLO model for person detection
yolo_model = YOLO('yolov8n.pt')

def calculate_angle(a, b, c):
    """Calculates the angle at point b formed by points a, b, c."""
    a_vec = np.array([a.x - b.x, a.y - b.y])
    c_vec = np.array([c.x - b.x, c.y - b.y])
    dot_product = np.dot(a_vec, c_vec)
    magnitude = np.linalg.norm(a_vec) * np.linalg.norm(c_vec)
    if magnitude == 0:
        return 0
    angle_rad = np.arccos(np.clip(dot_product / magnitude, -1.0, 1.0))
    return np.degrees(angle_rad)

def find_shooter(frame, yolo_model):
    """Uses YOLO to find the person most likely to be the shooter"""
    results = yolo_model(frame, verbose=False, conf=0.3)  # Lower confidence to 0.3

    people_boxes = []
    for result in results:
        if result.boxes is not None:
            for box in result.boxes:
                class_id = int(box.cls)
                if yolo_model.names[class_id] == 'person':
                    people_boxes.append(box.xyxy[0].cpu().numpy())

    if not people_boxes:
        return None

    frame_height, frame_width = frame.shape[:2]
    bottom_center = (frame_width // 2, frame_height)

    best_box = None
    min_distance = float('inf')

    for box in people_boxes:
        x1, y1, x2, y2 = box
        box_center = ((x1 + x2) / 2, (y1 + y2) / 2)
        distance = math.sqrt((box_center[0] - bottom_center[0])**2 +
                            (box_center[1] - bottom_center[1])**2)

        if distance < min_distance:
            min_distance = distance
            best_box = box

    return best_box

def smart_crop(frame, best_box, min_size=400):
    """
    Crops the frame around the detected person, but ensures the result
    is large enough for pose detection by resizing if necessary.
    """
    x1, y1, x2, y2 = best_box.astype(int)

    # Add generous padding (proportional to person size)
    person_height = y2 - y1
    person_width = x2 - x1
    padding_y = int(person_height * 1.2)  # 120% padding
    padding_x = int(person_width * 1.2)   # 120% padding

    x1 = max(0, x1 - padding_x)
    y1 = max(0, y1 - padding_y)
    x2 = min(frame.shape[1], x2 + padding_x)
    y2 = min(frame.shape[0], y2 + padding_y)

    cropped_frame = frame[y1:y2, x1:x2]

    # If cropping resulted in empty frame, return original frame
    if cropped_frame.size == 0:
        return frame

    # Resize if too small
    height, width = cropped_frame.shape[:2]
    if height < min_size or width < min_size:
        scale = max(min_size / height, min_size / width)
        new_height = int(height * scale)
        new_width = int(width * scale)
        cropped_frame = cv2.resize(cropped_frame, (new_width, new_height),
                                  interpolation=cv2.INTER_CUBIC)
        print(f"  Resized from {height}x{width} to {new_height}x{new_width}")

    return cropped_frame

# Load your CSV file
csv_path = './penalty_data/labels.csv'
labels_df = pd.read_csv(csv_path)
data_dir = './penalty_data'

# Lists to store our features and labels
feature_list = []
label_list = []
success_count = 0
fail_count = 0

print("Starting feature extraction from videos...")

# Loop through each row in the CSV
for index, row in labels_df.iterrows():
    video_name = row['video_name']
    frame_num = row['frame_number']
    direction_label = row['shooter_direction']

    video_path = os.path.join(data_dir, video_name)

    if not os.path.exists(video_path):
        print(f"Warning: Video file {video_path} not found. Skipping.")
        fail_count += 1
        continue

    # Open the video and seek to the specific frame
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
    success, original_frame = cap.read()
    cap.release()

    if not success:
        print(f"Could not read frame {frame_num} from {video_name}. Skipping.")
        fail_count += 1
        continue

    # Use YOLO to find the shooter
    shooter_box = find_shooter(original_frame, yolo_model)

    if shooter_box is None:
        print(f"No person detected in {video_name} at frame {frame_num}. Skipping.")
        fail_count += 1
        continue

    # Use SMART cropping
    processed_frame = smart_crop(original_frame, shooter_box, min_size=400)

    # Try pose detection on the processed frame
    with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.3) as pose:
        frame_rgb = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        if not results.pose_landmarks:
            print(f"No pose detected in processed frame of {video_name}:{frame_num}. Skipping.")
            fail_count += 1
            continue

        landmarks = results.pose_landmarks.landmark

        # --- FEATURE EXTRACTION ---
        try:
            # 1. Plant Foot Angles
            l_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE]
            l_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE]
            l_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
            plant_foot_angle_left = calculate_angle(l_ankle, l_knee, l_hip)

            r_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE]
            r_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE]
            r_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP]
            plant_foot_angle_right = calculate_angle(r_ankle, r_knee, r_hip)
        except (IndexError, AttributeError):
            plant_foot_angle_left = 0
            plant_foot_angle_right = 0

        # 2. Hip Orientation
        try:
            l_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
            l_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
            l_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE]
            shoulder_hip_angle = calculate_angle(l_shoulder, l_hip, l_knee)
        except (IndexError, AttributeError):
            shoulder_hip_angle = 0

        # 3. Torso Lean
        try:
            nose_y = landmarks[mp_pose.PoseLandmark.NOSE].y
            l_hip_y = landmarks[mp_pose.PoseLandmark.LEFT_HIP].y
            torso_lean = nose_y - l_hip_y
        except (IndexError, AttributeError):
            torso_lean = 0

        # 4. Hip-Shoulder Separation
        try:
            l_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
            r_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]
            l_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
            r_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP]

            shoulder_angle = math.degrees(math.atan2(r_shoulder.y - l_shoulder.y, r_shoulder.x - l_shoulder.x))
            hip_angle = math.degrees(math.atan2(r_hip.y - l_hip.y, r_hip.x - l_hip.x))
            hip_shoulder_separation = abs(shoulder_angle - hip_angle)
        except (IndexError, AttributeError):
            hip_shoulder_separation = 0

        # 5. Stance Width
        try:
            l_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE]
            r_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE]
            stance_width = abs(l_ankle.x - r_ankle.x)
        except (IndexError, AttributeError):
            stance_width = 0

        # 6. Foot Forwardness
        try:
            l_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE]
            r_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE]
            foot_forwardness = l_ankle.y - r_ankle.y
        except (IndexError, AttributeError):
            foot_forwardness = 0

        # 7. Body Lean
        try:
            l_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
            r_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]
            l_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
            r_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP]

            shoulder_center_x = (l_shoulder.x + r_shoulder.x) / 2
            hip_center_x = (l_hip.x + r_hip.x) / 2
            body_lean = shoulder_center_x - hip_center_x
        except (IndexError, AttributeError):
            body_lean = 0

        # Create feature vector
        features = [
            plant_foot_angle_left,
            plant_foot_angle_right,
            shoulder_hip_angle,
            torso_lean,
            hip_shoulder_separation,
            stance_width,
            foot_forwardness,
            body_lean
        ]

        # Append the ORIGINAL features and label
        feature_list.append(features)
        label_list.append(direction_label)
        success_count += 1
        print(f"✅ Processed {video_name}:{frame_num} -> {direction_label}")

        # --- DATA AUGMENTATION: Create MIRRORED version ---
        if direction_label == "left":
            mirrored_label = "right"
        elif direction_label == "right":
            mirrored_label = "left"
        else:
            mirrored_label = "center"

        # For mirroring, we swap left/right specific features and invert some
        mirrored_features = [
            plant_foot_angle_right,  # swapped
            plant_foot_angle_left,   # swapped
            shoulder_hip_angle,
            torso_lean,
            hip_shoulder_separation,
            stance_width,
            -foot_forwardness,       # inverted
            -body_lean               # inverted
        ]

        feature_list.append(mirrored_features)
        label_list.append(mirrored_label)
        print(f"✅ Created mirrored version -> {mirrored_label}")

print(f"\nExtraction complete! Results:")
print(f"✅ Successful extractions: {success_count}")
print(f"❌ Failed extractions: {fail_count}")
print(f"📊 Total samples: {len(feature_list)}")

# Convert to NumPy arrays
X = np.array(feature_list)
y = np.array(label_list)

print(f"\nFinal dataset shape: {X.shape}")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Training set: {X_train.shape}, Test set: {X_test.shape}")

# --- TRAIN FINAL MODEL ON TOP 5 FEATURES ---
# Use only the most important features
important_indices = [0, 1, 6, 7, 2]  # Plant_Foot_Left, Plant_Foot_Right, Foot_Forwardness, Body_Lean, Shoulder_Hip_Angle
X_reduced = X[:, important_indices]

# Split the reduced data
X_train_reduced, X_test_reduced, y_train_reduced, y_test_reduced = train_test_split(
    X_reduced, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Reduced dataset shape: {X_reduced.shape}")

# Train final KNN model (your best performer)
final_model = KNeighborsClassifier(n_neighbors=3)
final_model.fit(X_train_reduced, y_train_reduced)

# Evaluate
final_accuracy = final_model.score(X_test_reduced, y_test_reduced)
print(f"🎯 FINAL MODEL ACCURACY: {final_accuracy:.3f}")

# --- SAVE THE MODEL ---
final_model_filename = 'final_penalty_predictor.pkl'
joblib.dump(final_model, final_model_filename)
print(f"✅ Model saved as {final_model_filename}")

# Download the model to your computer
files.download(final_model_filename)
print("📦 Model downloaded! You can now use it for deployment.")

# --- CREATE LEFT/RIGHT ONLY CLASSIFIER ---
left_right_mask = (y == 'left') | (y == 'right')
X_lr = X_reduced[left_right_mask]
y_lr = y[left_right_mask]

lr_classifier = KNeighborsClassifier(n_neighbors=3)
lr_classifier.fit(X_lr, y_lr)
lr_accuracy = lr_classifier.score(X_lr, y_lr)
print(f"Left/Right only accuracy: {lr_accuracy:.3f}")

# Save this classifier too
lr_model_filename = 'left_right_predictor.pkl'
joblib.dump(lr_classifier, lr_model_filename)
files.download(lr_model_filename)

print("\n=== TRAINING COMPLETE ===")
print("You now have two models:")
print("1. final_penalty_predictor.pkl - For left/right/center prediction")
print("2. left_right_predictor.pkl - For high-confidence left/right prediction")

ERROR: Could not find a version that satisfies the requirement mediapipe==0.10.9 (from versions: 0.10.13, 0.10.14, 0.10.15, 0.10.18, 0.10.20, 0.10.21)
ERROR: No matching distribution found for mediapipe==0.10.9


100%|██████████| 6.23M/6.23M [00:00<00:00, 86.7MB/s]


UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL ultralytics.nn.tasks.DetectionModel was not an allowed global by default. Please use `torch.serialization.add_safe_globals([ultralytics.nn.tasks.DetectionModel])` or the `torch.serialization.safe_globals([ultralytics.nn.tasks.DetectionModel])` context manager to allowlist this global if you trust this class/function.

Check the documentation of to